## **모델 훈련 연습 문제**
___
- 출처 : 핸즈온 머신러닝 Ch04 연습문제 1, 5, 9, 10
- 개념 문제의 경우 텍스트 셀을 추가하여 정답을 적어주세요.

### **1. 수백만 개의 특성을 가진 훈련 세트에서는 어떤 선형 회귀 알고리즘을 사용할 수 있을까요?**

확률적 경사하강법, 미니배치 경사하강법
(계산복잡도가 커져서 경사하강법을 사용)
___


### **2. 배치 경사 하강법을 사용하고 에포크마다 검증 오차를 그래프로 나타내봤습니다. 검증 오차가 일정하게 상승되고 있다면 어떤 일이 일어나고 있는 걸까요? 이 문제를 어떻게 해결할 수 있나요?**

문제 이유 : 훈련 데이터에 너무 과적합된 경우, 또는 학습률이 너무 높은 경우   
해결법 : 정규화, 조기종료, 또는 학습률 낮추기
___

### **3. 릿지 회귀를 사용했을 때 훈련 오차가 검증 오차와 거의 비슷하고 둘 다 높았습니다. 이 모델에는 높은 편향이 문제인가요, 아니면 높은 분산이 문제인가요? 규제 하이퍼파라미터 $\alpha$를 증가시켜야 할까요 아니면 줄여야 할까요?**

과소적합(높은 편향, 낮은 분산)되었기 때문임.
편향이 높아서 생기는 모델 복잡도를 줄이기 위해 alpha값을 줄임.
___

### **4. 다음과 같이 사용해야 하는 이유는?**
- 평범한 선형 회귀(즉, 아무런 규제가 없는 모델) 대신 릿지 회귀 -> 과적합 방지 (L2 규제로 가중치 크기 제한)
- 릿지 회귀 대신 라쏘 회귀 -> 특성 선택이 필요할 때 (L1 규제로 일부 가중치를 0으로 만들어 불필요한 특성 제거)
- 라쏘 회귀 대신 엘라스틱넷 -> 균형 조절이 필요할 때 (릿지의 안정성 더하기)
___

### **추가) 조기 종료를 사용한 배치 경사 하강법으로 iris 데이터를 활용해 소프트맥스 회귀를 구현해보세요(사이킷런은 사용하지 마세요)**


---



In [1]:
# 데이터 불러오기
from sklearn.datasets import load_iris
iris=load_iris()

In [2]:
X = iris["data"][:, (2, 3)]  # 꽃잎 길이, 꽃잎 넓이
y = iris["target"]

In [4]:
# 모든 샘플에 편향 추가
import numpy as np
X_with_bias = np.c_[np.ones([len(X), 1]), X]

In [5]:
# train_test_split을 이용하지 않고 수동으로 나누기
test_ratio = 0.2
validation_ratio = 0.2
total_size = len(X_with_bias)

test_size = int(total_size * test_ratio)
validation_size = int(total_size * validation_ratio)
train_size = total_size - test_size - validation_size

rnd_indices = np.random.permutation(total_size)

X_train = X_with_bias[rnd_indices[:train_size]]
y_train = y[rnd_indices[:train_size]]
X_valid = X_with_bias[rnd_indices[train_size:-test_size]]
y_valid = y[rnd_indices[train_size:-test_size]]
X_test = X_with_bias[rnd_indices[-test_size:]]
y_test = y[rnd_indices[-test_size:]]

In [6]:
# 클래스 인덱스를 원핫 인코딩하는 함수 생성
def to_one_hot(y):
    n_classes = y.max() + 1
    m = len(y)
    Y_one_hot = np.zeros((m, n_classes))
    Y_one_hot[np.arange(m), y] = 1
    return Y_one_hot

In [7]:
y_train[:5]

array([2, 2, 2, 1, 0])

In [8]:
to_one_hot(y_train[:5])

array([[0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [9]:
# 훈련 세트와 테스트 세트의 타깃 클래스 확률을 담은 행렬 생성
Y_train_one_hot = to_one_hot(y_train)
Y_valid_one_hot = to_one_hot(y_valid)
Y_test_one_hot = to_one_hot(y_test)

In [10]:
# 소프트맥스 함수 생성
def softmax(logits):
    exps = np.exp(logits)
    exp_sums = np.sum(exps, axis=1, keepdims=True)
    return exps / exp_sums

In [11]:
# 입력과 출력의 개수 정의
n_inputs = X_train.shape[1]
n_outputs = len(np.unique(y_train))

In [12]:
# 학습기 훈련
eta = 0.01
n_iterations = 5001
m = len(X_train)
epsilon = 1e-7

Theta = np.random.randn(n_inputs, n_outputs)

for iteration in range(n_iterations):
    logits = X_train.dot(Theta)
    Y_proba = softmax(logits)
    if iteration % 500 == 0:
        loss = -np.mean(np.sum(Y_train_one_hot * np.log(Y_proba + epsilon), axis=1))
        print(iteration, loss)
    error = Y_proba - Y_train_one_hot
    gradients = 1/m * X_train.T.dot(error)
    Theta = Theta - eta * gradients

0 4.399594500711989
500 1.0077649706491831
1000 0.7979185791686351
1500 0.6759923949949204
2000 0.5997514855711266
2500 0.5477224794986055
3000 0.5095182592435085
3500 0.47985708264153243
4000 0.45584934692295903
4500 0.43579865306005316
5000 0.4186460561831074


In [13]:
# 파라미터 확인
Theta

array([[ 4.06880908,  0.07868178, -2.10493949],
       [-1.16675199,  0.52714367,  0.30685292],
       [-0.12322622, -0.60900251,  1.42807626]])

In [14]:
# 예측 정확도 확인
logits = X_valid.dot(Theta)
Y_proba = softmax(logits)
y_predict = np.argmax(Y_proba, axis=1)

accuracy_score = np.mean(y_predict == y_valid)
accuracy_score

0.9666666666666667